We start by importing some libraries that we will use. Note that we prefer using the GPU so make sure the runtime is set to one of the GPU environments. The "free" T4 seems sufficient here.

In [1]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
import gc
import os


The first function calculates the yield given the current value of  lt  and  ft . Note that we are using  lt  here instead of  αt . This is also utilizing pytorch and works best on the GPU. Comments:

Could try to see if we could "save" some memmory. It will be quite memory intensive when using automatic differentiation
Need to verify that it is correct. I have checked the convergence towards the short rate and it looks fine

In [2]:
def getYield(f, l, tau, pars, M, dt, device):
    rhoA, rhoB, nu, DEL, kap, lbar, sig_l, muY, sigY = pars

    NT = round(tau / dt) #Number of time steps

    # Initialize tensors for all paths
    ft = f * torch.ones(M, device=device)
    lt = l * torch.ones(M, device=device)
    DFt = torch.ones(M, device=device)

    # Generate random numbers for all paths and time steps
    dZ = torch.sqrt(torch.tensor(dt)) * torch.randn(NT, M, device=device)

    for i in range(NT):
        theta_alp = DEL * (0.5 - ft)
        alpt = 1 / (1 + torch.exp(-lt))
        phit = ft / (rhoA + nu) + (1 - ft) / (rhoB + nu)
        betAt = (rhoA + nu) * phit
        betBt = (rhoB + nu) * phit

        muft = nu * (alpt * betAt * (1 - ft) - (1 - alpt) * betBt * ft) + (rhoB - rhoA) * ft * (1 - ft) + DEL**2 * (1/2 - ft) * ft * (1 - ft)
        sigft = ft * (1 - ft) * DEL
        dft = muft * dt + sigft * (dZ[i, :] - theta_alp * dt)
        dlt = kap * (lbar - lt) * dt + sig_l * (dZ[i, :] - theta_alp * dt)

        lt =lt + dlt  # update l0
        ft =ft + dft  # update f0
        rbar = muY - sigY**2
        r = rbar + rhoA * ft + rhoB * (1 - ft)  + nu * (1 - alpt * betAt - (1 - alpt) * betBt)
        DFt *= torch.exp(-r * dt)

    B = torch.mean(DFt)
    return -torch.log(B) / tau

In [3]:
def getYieldandYieldVola(f, alp, tau, pars, M, dt, device):
  rhoA, rhoB, nu, DEL, kap, lbar, sig_l, muY, sigY = pars
  #Since we are doing autodiff we need to make sure we have enabled autograd
  l = - -np.log(1/alp-1)
  lt = torch.tensor([l], device=device, requires_grad=True)  # Example value
  ft = torch.tensor([f], device=device, requires_grad=True)  # Example value
  #Calculating the yield
  y = getYield(ft, lt, tau, pars, M, dt, device)
  # Compute gradients
  y.backward()
  sigY = ft.grad.item()*ft*(1-ft)*DEL+lt.grad.item()*sig_l
  return y.item(), sigY.item(), ft.grad.item(), lt.grad.item()

In [4]:
def getShortRate(f, alp, tau, pars):
  rhoA, rhoB, nu, DEL, kap, lbar, sig_l, muY, sigY = pars
  phiA = 1/(rhoA + nu)
  phiB = 1/(rhoB + nu)
  phi = f*phiA + (1-f)*phiB
  betA = (rhoA+nu)*phi
  betB = (rhoB+nu)*phi
  bet = alp*betA+(1-alp)*betB
  rhot = f*rhoA+(1-f)*rhoB
  r = rhot + muY-sigY**2 + nu*(1-bet)
  return r

In [5]:
def getShortRateVola(f, alp, tau, pars):
  rhoA, rhoB, nu, DEL, kap, lbar, sig_l, muY, sigY = pars
  phiA = 1/(rhoA + nu)
  phiB = 1/(rhoB + nu)
  phi = f*phiA + (1-f)*phiB
  betA = (rhoA+nu)*phi
  betB = (rhoB+nu)*phi
  Delrho = rhoB-rhoA
  sigalp = alp*(1-alp)*sig_l
  sigf = DEL*f*(1-f)
  J = nu*(alp*(rhoA+nu)+(1-alp)*(rhoB+nu))
  k = -Delrho-J*(1/(rhoA+nu) - 1/(rhoB+nu))
  sigr = nu*Delrho*phi*sigalp + k*sigf
  return sigr


Here we set up the specific environment for PyTorch

In [6]:
# Set up device - will use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Uncomment the line below to force CPU usage if needed
# device = torch.device("cpu")
print("Device:", device)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

Device: cuda
CUDA available: True
GPU name: NVIDIA GeForce RTX 4090


In [7]:
DEL = 0.8
rhoA = -0.015
rhoB = 0.025
nu = 0.02
kap = 0.01
lbar = 0
sig_l = 0.1
muY = 0.02
sigY = 0.033
pars = [rhoA , rhoB, nu, DEL, kap, lbar, sig_l, muY, sigY]

In [8]:
dt = 1 / 12
M = 1000  # M = 500000 in the paper

In [9]:
taus = [dt, 1, 2, 3, 4, 5, 7, 10]  # Example tau values
f_values = np.linspace(0.01, 0.99, 200)  # f values from 0.01 to 0.99
alp = 0.5
plt.figure(figsize=(10, 6))
r_values = []
for f in f_values:
        r = getShortRate(f, alp, 1, pars)
        r_values.append(r)
y_values_by_tau = {}
BondRetStd_by_tau = {}
RxValues_by_tau = {}
sigrValues_by_tau = {}
sigyValues_by_tau = {}
for tau in taus:
    y_values = []
    bondRet = []
    RxValues = []
    sigrValues = []
    sigyValues = []
    for f in f_values:
        y, sigy, dydf, dydl = getYieldandYieldVola(f, alp, tau, pars, M, dt, device)
        sigr =  getShortRateVola(f, alp, tau, pars)
        sigyValues.append(sigy)
        sigrValues.append(sigr)
        bondStd = tau*np.sqrt(np.sum(sigy**2))
        rx = -sigy*tau*DEL*(0.5-f)
        RxValues.append(rx)
        r = getShortRate(f, alp, tau, pars)
        y_values.append(y)
        bondRet.append(bondStd)
    BondRetStd_by_tau[tau] = bondRet
    RxValues_by_tau[tau] = RxValues
    sigrValues_by_tau[tau] = sigrValues
    sigyValues_by_tau[tau] = sigyValues
    # Store the y_values for the current tau
    y_values_by_tau[tau] = y_values

<Figure size 1000x600 with 0 Axes>

In [10]:
# Ensure the directory exists
os.makedirs('Data/Model Disagreement', exist_ok=True)

# Save the data
np.savez('Data/Model Disagreement/TEST_model_data_for_f_plots.npz',
         pars=pars,
         taus=taus,
         f_values=f_values,
         alp=alp,
         BondRetStd_by_tau=BondRetStd_by_tau,
         RxValues_by_tau=RxValues_by_tau,
         sigrValues_by_tau=sigrValues_by_tau,
         sigyValues_by_tau=sigyValues_by_tau,
         y_values_by_tau=y_values_by_tau)

print("Data saved to 'Data/Model Disagreement/TEST_model_data_for_f_plots.npz'")


Data saved to 'Data/Model Disagreement/TEST_model_data_for_f_plots.npz'
